MA线（均线）：MA线指的是移动平均线，它是由一段时间的开盘价(也可以是收盘价)之和除以这段时间的交易日总天数得到的数据连接成一条线得到的，例如日线MA5指的是前5个交易日的开盘价的和除以5。

In [2]:
!pip install yfinance

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as webdata
import datetime

In [4]:
yf.pdr_override()
start_day = datetime.datetime(2000, 1, 1)
end_day = datetime.datetime(2022, 12, 31)
stock_code = input("美股直接输入股票代码如GOOG \n港股输入代码+对应股市，如腾讯：0700.hk \n国内股票需要区分上证和深证，股票代码后面加.ss或者.sz\n请输入你要查询的股票代码：")
stock_info = webdata.get_data_yahoo(stock_code, start_day, end_day)
print(stock_info.head())

美股直接输入股票代码如GOOG 
港股输入代码+对应股市，如腾讯：0700.hk 
国内股票需要区分上证和深证，股票代码后面加.ss或者.sz
请输入你要查询的股票代码：600519.ss
[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2001-08-27  6.867947  7.518720  6.537585  7.074920   5.239724  204166311
2001-08-28  6.963473  7.363490  6.887848  7.335628   5.432805   65145307
2001-08-29  7.359509  7.363490  7.184378  7.240101   5.362058   26758394
2001-08-30  7.220200  7.464986  7.164476  7.383391   5.468178   24125560
2001-08-31  7.393342  7.486878  7.323687  7.365480   5.454914   11673333


In [5]:
stock_info.to_csv("600519.csv")

In [6]:
df = pd.read_csv("600519.csv", index_col="Date",parse_dates=['Date'])[["Open","High","Low","Close"]]
df

,Open,High,Low,Close
Date,,,,
2001-08-27,6.867947,7.518720,6.537585,7.074920
2001-08-28,6.963473,7.363490,6.887848,7.335628
2001-08-29,7.359509,7.363490,7.184378,7.240101
2001-08-30,7.220200,7.464986,7.164476,7.383391
2001-08-31,7.393342,7.486878,7.323687,7.365480
...,...,...,...,...
2022-12-26,1771.000000,1771.000000,1735.020020,1742.060059
2022-12-27,1738.000000,1747.150024,1725.500000,1733.000000
2022-12-28,1745.880005,1747.000000,1708.010010,1733.000000


In [7]:
df = df['2015-01':]
df_month = df.resample("MS").first()  # 每月的第一个交易日
df_year = df.resample("A").last()[:-1] # 每年最后一个交易日
df_year

,Open,High,Low,Close
Date,,,,
2015-12-31,218.220001,219.500000,217.820007,218.190002
2016-12-31,326.609985,335.290009,326.609985,334.149994
2017-12-31,718.000000,726.500000,691.599976,697.489990
2018-12-31,563.299988,596.400024,560.000000,590.010010
2019-12-31,1183.000000,1188.000000,1176.510010,1183.000000
2020-12-31,1941.000000,1998.979980,1941.000000,1998.000000
2021-12-31,2070.000000,2072.979980,2028.000000,2050.000000


In [20]:
"""这个策略是从2015-2021年
每年的收益 = 每年最后一天开盘价* total_stoc - cost_year:(取每月第一天的开盘价相加) * total_stock(12个月 * 100 )
最后收益想加，花费相加
"""
total_cost = 0
year_cost = 0
total_stock = 0
make_money = 0
for y in range(2015,2022):
  year_cost = df_month["{}".format(y)]['Open'].sum() * 100
  total_stock = len(df_month["{}".format(y)]['Open']) * 100
  print(f'股票数量：{total_stock}')
  if y != 2021:
    make_money += df_year["{}".format(y)]['Open'][0] * total_stock - year_cost
    total_stock = 0
    total_cost += year_cost
    year_cost = 0

make_money += total_stock * df["Open"][-1] -year_cost
total_cost += year_cost

print("从2015年至今一共赚了:", round(make_money, 2))
print("从2015年至今一共花了:", round(total_cost, 2))

股票数量：1200
股票数量：1200
股票数量：1200
股票数量：1200
股票数量：1200
股票数量：1200
股票数量：1200
从2015年至今一共赚了: 916867.15
从2015年至今一共花了: 7106488.82


<ipython-input-20-11c0d2287d59>:6: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  year_cost = df_month["{}".format(y)]['Open'].sum() * 100
<ipython-input-20-11c0d2287d59>:7: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  total_stock = len(df_month["{}".format(y)]['Open']) * 100
<ipython-input-20-11c0d2287d59>:10: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  make_money += df_year["{}".format(y)]['Open'][0] * total_stock - year_cost


### 双均线策略
金叉：短期均线上穿长期均线，买入信号

死叉：短期均线下穿长期均线，卖出信号

In [10]:
df['ma5'],df['ma30'] = np.nan, np.nan
for i in range(4, len(df)):
  df.loc[df.index[i],'ma5'] = df["Open"][i-4:i+1].mean()

for i in range(29, len(df)):
  df.loc[df.index[i],'ma30'] = df['Open'][i-29:i+1].mean()

df

<ipython-input-10-73e96f264bd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ma5'],df['ma30'] = np.nan, np.nan


,Open,High,Low,Close,ma5,ma30
Date,,,,,,
2015-01-05,174.545456,185.672729,173.636368,184.109085,NaN,NaN
2015-01-06,181.818176,184.145447,178.199997,179.845459,NaN,NaN
2015-01-07,178.218185,181.363632,172.718185,175.399994,NaN,NaN
2015-01-08,176.363632,176.836365,172.854538,174.327271,NaN,NaN
2015-01-09,173.090912,177.972733,172.827271,173.009094,176.807272,NaN
...,...,...,...,...,...,...
2022-12-26,1771.000000,1771.000000,1735.020020,1742.060059,1753.885986,1653.313668
2022-12-27,1738.000000,1747.150024,1725.500000,1733.000000,1748.419995,1660.180334
2022-12-28,1745.880005,1747.000000,1708.010010,1733.000000,1752.795996,1665.676335


In [18]:
jin_cross = []
si_cross = []
for i in range(0,len(df)):
  if df['ma5'][i] >= df['ma30'][i] and df['ma5'][i-1] < df['ma30'][i-1]:
    jin_cross.append(df.index[i])
  elif df['ma5'][i] <= df['ma30'][i] and df['ma5'][i-1] > df['ma30'][i-1]:
    si_cross.append(df.index[i])

In [20]:
principal = 100000
total_stock = 0
seris1 = pd.Series(1, index=jin_cross)
seris2 = pd.Series(0, index=si_cross)

myseries = seris1.append(seris2).sort_index()

for i in range(0, len(myseries)):
  stock_price = df['Open'][myseries.index[i]]
  if myseries.iloc[i] == 1:
    buy_stock = int(principal/(stock_price*100))
    principal -= buy_stock* stock_price * 100
    total_stock += buy_stock * 100
  else:
    principal += total_stock * stock_price
    total_stock = 0

stock_price = df['Open'][-1]
principal += total_stock * stock_price
print("从2015至年共赚了：",round(principal-100000,2))

从2015至年共赚了： 792777.62


<ipython-input-20-e2dc776e9b61>:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  myseries = seris1.append(seris2).sort_index()


### 参考：https://zhuanlan.zhihu.com/p/412248860